In [1]:
# get spark
import findspark
findspark.init('/usr/local/Cellar/apache-spark/2.4.5/libexec')

In [2]:
# import required files
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature  import RegexTokenizer,StopWordsRemover,CountVectorizer
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
import pandas as pd
from pyspark.sql.functions import lower
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [3]:
sc = pyspark.SparkContext()

In [4]:
# create spark config
spark = SparkSession \
       .builder \
       .appName("Proj3") \
       .config("spark.driver.memory", "4g") \
       .config("spark.executor.memory", "4g") \
       .getOrCreate() 

In [5]:
# read the input data (training, mapping and testing)
train_dframe = pd.read_csv('train.csv')
map_dframe = pd.read_csv('mapping.csv')
test_dframe = pd.read_csv('test.csv')
train_data = spark.createDataFrame(train_dframe)
mapping_data = spark.createDataFrame(map_dframe,['id','genre'])
test_data = spark.createDataFrame(test_dframe)

In [6]:
# Display the details of input data
train_data.show()
test_data.show()
genre_count = mapping_data.count()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [7]:
# remove stopwords and transform training data into required format
tokenizer = RegexTokenizer(inputCol = "plot", outputCol = "ptok",pattern="\\w+", gaps=False)
train_data = tokenizer.transform(train_data)
remover = StopWordsRemover(inputCol = "ptok", outputCol = "clr")
train_data = remover.transform(train_data)
word_count = CountVectorizer(inputCol = "clr", outputCol = "feature",vocabSize=1000)
model = word_count.fit(train_data)
train_data = model.transform(train_data)
train_data.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|                ptok|                 clr|             feature|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|(1000,[10,129,186...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|(1000,[2,6,7,10,1...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|(1000,[1,3,8,10,1...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[the, lemon, drop...|[lemon, drop, kid...|(1000,[7,9,12,15,...|
|  595909|   A Cry in the Dark|Seventh-da

In [8]:
# transform test data into required format
test_data = tokenizer.transform(test_data)
test_data = remover.transform(test_data)
test_data = model.transform(test_data)
test_data.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|                ptok|                 clr|             feature|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|(1000,[0,3,4,7,8,...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|(1000,[7,8,56,71,...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|(1000,[5,7,11,12,...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|(1000,[0,66,81,86...|
|24165951|        Getting Even|A soldier-of-fort...|[a, soldier, of, ...|[soldier, fortune...|(1000,[18,379,433...|
| 1925869|  River of No Return|Set in the Northw...|[set, in, the, no...

In [9]:
# multilable classification for genre
mapper_details = mapping_data.select("genre","id").rdd.collectAsMap()

def genreMapSplit(mapper_obj):
    result = []
    for element in mapper_obj[1:-1].split(","):
        result.append(mapper_details.get(element.strip()[1:-1]))
    result.sort()
    return result

def resultMapLabel(mapper_obj):
    result = []
    for index in range(0,genre_count):
        result.append(0)
    for index in mapper_obj:
        result[index] = 1
    return result

# Mapped data
udf_genre = f.udf(genreMapSplit,ArrayType(IntegerType()))
train_data = train_data.withColumn("mapped",udf_genre("genre"))
train_data.select("feature","genre","mapped").show()

# Labelled data
udf_label = f.udf(resultMapLabel,ArrayType(IntegerType()))
train_data = train_data.withColumn("label",udf_label("mapped"))
train_data.select("label").show()

+--------------------+--------------------+----------------+
|             feature|               genre|          mapped|
+--------------------+--------------------+----------------+
|(1000,[10,129,186...|['World cinema', ...|          [0, 5]|
|(1000,[2,6,7,10,1...|['Action/Adventur...|  [0, 4, 10, 17]|
|(1000,[1,3,8,10,1...|['Musical', 'Acti...|      [0, 4, 16]|
|(1000,[7,9,12,15,...|          ['Comedy']|             [1]|
|(1000,[2,8,9,14,1...|['Crime Fiction',...|       [0, 5, 6]|
|(1000,[2,4,5,12,1...|['Action/Adventur...|   [0, 3, 4, 10]|
|(1000,[0,1,2,3,4,...|['Thriller', 'Dra...|       [0, 3, 7]|
|(1000,[0,1,2,5,7,...|           ['Drama']|             [0]|
|(1000,[0,4,9,17,3...|['Black-and-white...|[0, 1, 2, 8, 19]|
|(1000,[11,31,38,6...|['Animation', 'Sh...|    [12, 13, 15]|
|(1000,[1,3,4,7,9,...|          ['Comedy']|             [1]|
|(1000,[0,2,14,32,...|['Crime Fiction',...|    [0, 1, 5, 6]|
|(1000,[4,5,15,17,...|          ['Comedy']|             [1]|
|(1000,[0,1,2,3,6,...|  

In [10]:
# final output
final_result = []
for index in range(0, genre_count):
    def parsePoint(row):
        return LabeledPoint(row.label[index], MLLibVectors.fromML(row.feature))
    labelled_output = train_data.rdd.map(parsePoint)
    modelpred = LogisticRegressionWithLBFGS.train(labelled_output)
    pred = test_data.rdd.map(lambda t: (t.movie_id, modelpred.predict(MLLibVectors.fromML(t.feature))))
    final_result.append(pred.collect())
    print(f"Finished processing genere {index}")

final_result_obj = {}
for element in final_result:
    for temp in element:
        if temp[0] not in final_result_obj:
            final_result_obj[temp[0]]=[]
            final_result_obj[temp[0]].append(int(temp[1]))
        else:
            final_result_obj[temp[0]].append(int(temp[1]))

for key in final_result_obj:
    print(key,final_result_obj[key])

Finished processing genere 0
Finished processing genere 1
Finished processing genere 2
Finished processing genere 3
Finished processing genere 4
Finished processing genere 5
Finished processing genere 6
Finished processing genere 7
Finished processing genere 8
Finished processing genere 9
Finished processing genere 10
Finished processing genere 11
Finished processing genere 12
Finished processing genere 13
Finished processing genere 14
Finished processing genere 15
Finished processing genere 16
Finished processing genere 17
Finished processing genere 18
Finished processing genere 19
1335380 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
29062594 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9252321 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13455076 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24165951 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
1925869 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

2456941 [1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
27359507 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
10332554 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10995901 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19496914 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21099947 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21654511 [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
9198608 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
6764425 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5988501 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4989559 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
29434234 [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
25434917 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5316735 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
16967207 [0, 1, 0, 0, 1, 0

21946062 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2812897 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
25447923 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
25445821 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7601868 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32076153 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7503887 [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]
16523191 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30865545 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9919820 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20333975 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6324060 [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
14222317 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
13141411 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
25845778 [1, 0, 0, 0, 0, 

31230287 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24041290 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
1205454 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
33447795 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2249957 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7145079 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0]
6838440 [1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28870239 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3937957 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
28247074 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
565758 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
26191454 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13490092 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
15570371 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17465733 [1, 0, 0, 0, 0, 0,

1811784 [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
33612606 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
7660132 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
25989167 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6352734 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4542637 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
26675487 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1645598 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
12689587 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
31941863 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
23633579 [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
24668730 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
8042486 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24817139 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4186781 [1, 0, 0, 0, 1, 0,

1683515 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20376273 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
16781552 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8770396 [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13180264 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18514527 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
14769428 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
12619383 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4110642 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2063141 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3751460 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18926657 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
88609 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
10026747 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10934134 [1, 0, 0, 1, 1, 1, 

In [11]:
# csv files
df = pd.DataFrame.from_dict(final_result_obj,orient='index')
df.index.names = ['movie_id']
df['predictions'] = df[df.columns[:]].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
df.to_csv('final_csv_ans_part2.csv',columns = ['predictions'])